In [ ]:
import nnabla as nn

import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
from nnabla.monitor import tile_images

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 64
Z_dim = 100
eps_dim = 4
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
lr = 1e-3

In [ ]:
# Q(z|X, eps) Encoder
def Q(X, eps, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("Q"):  # Parameter scope can be nested
        h = F.concatenate(X, eps)
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("last_layer"):
            z = PF.affine(h, Z_dim)
    return z

In [ ]:
# P(X|z) Decoder
def P(z, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("P"):  # Parameter scope can be nested
        h = z
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("last_layer"):
            X = F.sigmoid(PF.affine(h, X_dim))
    return X

In [ ]:
# Discriminator
def T(X, z, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("T"):  # Parameter scope can be nested
        h = z
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("classifier"):
            y = PF.affine(h, 1)
    return y

In [ ]:
Q_solver = S.Adam(lr)
with nn.parameter_scope("Q"):
    Q_solver.set_parameters(nn.get_parameters()) 

P_solver = S.Adam(lr)
with nn.parameter_scope("P"):
    P_solver.set_parameters(nn.get_parameters())   

T_solver = S.Adam(lr)
with nn.parameter_scope("T"):
    T_solver.set_parameters(nn.get_parameters())

In [ ]:
def reset_grad():
    Q_solver.zero_grad()
    P_solver.zero_grad()
    T_solver.zero_grad()

In [ ]:
def sample_X(size, include_y=False):
    X, y = mnist.train.next_batch(size)
    X = nn.Variable.from_numpy_array(X)

    if include_y:
        y = np.argmax(y, axis=1).astype(np.int)
        y = nn.Variable.from_numpy_array(y)
        return X, y

    return X

In [ ]:
def show16(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    plt.show()

In [ ]:
for it in range(100000):
    X = sample_X(mb_size)
    eps = nn.Variable.from_numpy_array(np.random.randn(mb_size, eps_dim))
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    
    # Optimize VAE
    z_sample = Q(X, eps)
    X_sample = P(z_sample)
    T_sample = T(X, z)

    disc = F.mean(-T_sample)
    loglike = - F.sum(F.binary_cross_entropy(X_sample, X) / mb_size)
    
    elbo = -(disc + loglike)
    
    elbo.forward()
    Q_solver.update()
    P_solver.update()
    reset_grad()
    
    
    # Discriminator T Update
    z_sample = Q(X, eps)
    T_q = F.sigmoid(T(X, z_sample))
    T_prior = F.sigmoid(T(X, z))
    
    T_loss = - F.mean(F.log(T_q + 1e-8) + F.log(1. - T_prior + 1e-8))
    
    T_loss.forward()
    T_loss.backward()
    T_solver.update()
    reset_grad()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Step: {}, ELBO: {:.4}, T_loss: {:.4}'.format(it, -elbo.d, -T_loss.d))
        
        samples = P(z).d[:16]
        show16(samples)